In [0]:
df = spark.sql("""
    with cte as (
        select 
            location,
            year,
            month,
            round(avg(case when mobility_retail = 0 then null else mobility_retail end), 2) as avg_retail,
            round(avg(case when mobility_grocery = 0 then null else mobility_grocery end), 2) as avg_grocery,
            round(avg(case when mobility_workplace = 0 then null else mobility_workplace end), 2) as avg_workplace,
            round(avg(case when mobility_home = 0 then null else mobility_home end), 2) as avg_home
        from global_temp.covidcombined
        group by location, year, month
    )
    
    select *,
        dense_rank() over (partition by location, year order by avg_workplace asc) as workplace_reduction_rank,
        dense_rank() over (partition by location, year order by avg_home desc) as home_stay_rank
    from cte
    order by location, year, month
""")

In [0]:
df_2 = df.fillna({
    "avg_retail": -1,
    "avg_grocery": -1,
    "avg_workplace": -1,
    "avg_home": -1
})

In [0]:
df_2.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/gold/monthly_mobility_summary")

In [0]:
spark.catalog.uncacheTable("global_temp.covidcombined")
spark.catalog.dropGlobalTempView("covidcombined")